In [0]:
!wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb

--2024-03-12 11:28:24--  http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
Resolving archive.ubuntu.com (archive.ubuntu.com)... 91.189.91.82, 91.189.91.81, 185.125.190.36, ...
Connecting to archive.ubuntu.com (archive.ubuntu.com)|91.189.91.82|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1318204 (1.3M) [application/vnd.debian.binary-package]
Saving to: ‘libssl1.1_1.1.1f-1ubuntu2_amd64.deb.1’

libssl1.1_1.1.1f-1u 100%[===================>]   1.26M  --.-KB/s    in 0.1s    

2024-03-12 11:28:25 (11.2 MB/s) - ‘libssl1.1_1.1.1f-1ubuntu2_amd64.deb.1’ saved [1318204/1318204]

Selecting previously unselected package libssl1.1:amd64.
(Reading database ... 98474 files and directories currently installed.)
Preparing to unpack libssl1.1_1.1.1f-1ubuntu2_amd64.deb ...
Unpacking libssl1.1:amd64 (1.1.1f-1ubuntu2) ...
Setting up libssl1.1:amd64 (1.1.1f-1ubuntu2) ...
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialo

In [0]:
%pip install azure-cognitiveservices-speech==1.33.0

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 18.0 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
#!pip install -U azure-cognitiveservices-speech

In [0]:
dbutils.library.restartPython()

In [0]:
import os
import time
import azure.cognitiveservices.speech as speechsdk
import datetime
import json
from pathlib import Path

In [0]:
os.environ['SPEECH_KEY'] = "bb33fb8ed4e54099b7e845ca3508bf26"
os.environ['SPEECH_REGION'] = "eastus2"

In [0]:
print(os.environ.get('SPEECH_KEY'))
print(os.environ.get('SPEECH_REGION'))

bb33fb8ed4e54099b7e845ca3508bf26
eastus2


In [0]:
#Call back functions

def conversation_transcriber_recognition_canceled_cb(evt: speechsdk.SessionEventArgs):
    cancellation_details = evt.cancellation_details
    print('Transciption session cancelled')
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        print("Error details: {}".format(cancellation_details.error_details))

def conversation_transcriber_session_stopped_cb(evt: speechsdk.SessionEventArgs):
    print('Transciption session stopped')

def conversation_transcriber_session_started_cb(evt: speechsdk.SessionEventArgs):
    print('Transciption session started')


In [0]:
def transcribe_audio(filename):
    transcriptions_list = []
    # This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
    speech_config = speechsdk.SpeechConfig(subscription=os.environ.get('SPEECH_KEY'), region=os.environ.get('SPEECH_REGION'))
    speech_config.request_word_level_timestamps()
    speech_config.speech_recognition_language="en-GB"
    speech_config.output_format = speechsdk.OutputFormat(1)

    audio_config = speechsdk.audio.AudioConfig(filename=filename)
    conversation_transcriber = speechsdk.transcription.ConversationTranscriber(speech_config=speech_config, audio_config=audio_config)

    transcribing_stop = False

    def conversation_transcriber_transcribed_cb(evt: speechsdk.SpeechRecognitionEventArgs):
        transcriptions_list.append(evt)

    def stop_cb(evt: speechsdk.SessionEventArgs):
        #"""callback that signals to stop continuous recognition upon receiving an event `evt`"""
        nonlocal transcribing_stop
        transcribing_stop = True

    # Connect callbacks to the events fired by the conversation transcriber
    conversation_transcriber.transcribed.connect(conversation_transcriber_transcribed_cb)
    conversation_transcriber.session_started.connect(conversation_transcriber_session_started_cb)
    conversation_transcriber.session_stopped.connect(conversation_transcriber_session_stopped_cb)
    conversation_transcriber.canceled.connect(conversation_transcriber_recognition_canceled_cb)
    # stop transcribing on either session stopped or canceled events
    conversation_transcriber.session_stopped.connect(stop_cb)
    conversation_transcriber.canceled.connect(stop_cb)

    conversation_transcriber.start_transcribing_async()

    # Waits for completion.
    while not transcribing_stop:
        time.sleep(.5)

    conversation_transcriber.stop_transcribing_async()

    return transcriptions_list

In [0]:
def convert_time_delta_to_string(time_delta):
    hours, remainder = divmod(time_delta.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = time_delta.microseconds // 1000

    formatted_time = f"{hours:02}:{minutes:02}:{seconds:02}.{milliseconds:02}"
    
    return formatted_time

In [0]:
def ticks_to_timedelta(tick):
    microsecond = tick / 10
    return datetime.timedelta(microseconds=microsecond)

In [0]:
def process_transcription_json(data):
    string_data = []
    for i in range(len(data)):
        json_data = json.loads(data[i].result.json)
        start_time = ticks_to_timedelta(json_data['Offset'])
        end_time = ticks_to_timedelta(json_data['Offset'] + json_data['Duration'])
        speaker = json_data['SpeakerId']
        text = json_data['DisplayText']

        start_time_string = convert_time_delta_to_string(start_time)
        end_time_string = convert_time_delta_to_string(end_time)

        string_data.append(f"({start_time_string} - {end_time_string}) Speaker [{speaker}] : {text}\n")
    
    string = "".join(string_data)
    return string

In [0]:
base_path = "/FileStore/Viiv_audio"
folder_path = "test"
CALL_RECORDINGS = []
for file in dbutils.fs.ls(base_path):
  if file.isDir():
    continue
  CALL_RECORDINGS.append(os.path.splitext(file.name)[0])
  print(file.name)

09_27_2023_PA_PrEP_Consumer_Typing_Research_HCP_2.mp4
09_27_2023_PCP_PrEP_Consumer_Typing_Research_HCP_1.mp4
09_27_2023_PCP_PrEP_Consumer_Typing_Research_HCP_3.mp4
09_29_2023_PA_PrEP_Consumer_Typing_Research_HCP__4.m4a
09_29_2023_PA_PrEP_Consumer_Typing_Research_HCP__5.m4a
10_02_2023_ObGyn_PrEP_Consumer_Typing_Research_HCP__7.m4a
10_02_2023_ObGyn_PrEP_Consumer_Typing_Research__9.m4a
10_02_2023_PA_PrEP_Consumer_Typing_Research_HCP__8.m4a
10_02_2023_PCP_PrEP_Consumer_Typing_Research_HCP__10.m4a
10_03_2023_ObGyn_PrEP_Consumer_Typing_Research__12.m4a
10_03_2023_PCP_PrEP_Consumer_Typing_Research__13.m4a
10_04_2023_NP_PrEP_Consumer_Typing_Research__15.m4a
10_04_2023_ObGyn_PrEP_Consumer_Typing_Research__14.m4a
10_05_2023_NP_PrEP_Consumer_Typing_Research__17.m4a
10_05_2023_PCP_PrEP_Consumer_Typing_Research__16.m4a
Vaporetto_E_US_NAVIGATOR_5_Rosita_17Nov_5pmET_PatientAdvocate.wav
Vaporetto_E_US_PAT11_Rosita_8Nov_12pmET_Patient.wav
Vaporetto_E_US_PAT17_Rosita_13Nov_11amET_Patient.wav
Vaporetto_E

In [0]:
base_path="/dbfs/FileStore/Viiv_audio"
input_relative_path = f"{base_path}/00_audio/{folder_path}"
output_relative_path = f"{base_path}/00_transcripts/{folder_path}"

for file_name in sorted(os.listdir(input_relative_path)):
        if any((match := x) in file_name for x in CALL_RECORDINGS):
            if file_name.endswith(".wav"):
                name = os.path.splitext(file_name)[0]
                

                output_path = f'{output_relative_path}/{match}_transcript.txt'

                if os.path.exists(output_path):
                    print(f"Skipping {name} as it already exists")
                    continue
                else:
                    print(f"Processing {name}")
                    try:
                        transcripts = transcribe_audio(f'{input_relative_path}/{file_name}')
                        with open(output_path, "w") as f:
                                f.write(process_transcription_json(transcripts))

                    except Exception as err:
                        print("Encountered exception. {}".format(err))

Skipping Vaporetto_E_US_NAVIGATOR_5_Rosita_17Nov_5pmET_PatientAdvocate as it already exists
Skipping Vaporetto_E_US_PAT11_Rosita_8Nov_12pmET_Patient-1 as it already exists
Skipping Vaporetto_E_US_PAT11_Rosita_8Nov_12pmET_Patient as it already exists
Skipping Vaporetto_E_US_PAT17_Rosita_13Nov_11amET_Patient as it already exists
Skipping Vaporetto_E_US_PAT18_Rosita_14_Nov_615pmET_NonUser as it already exists
Skipping Vaporetto_E_US_PAT19_Rosita_14Nov_11amET_PREP as it already exists
Skipping Vaporetto_E_US_PAT20_Rosita_14_Nov_1215pmET_Prep as it already exists
Skipping Vaporetto_E_US_PAT22_Rosita_15_Nov_4pmET_Patient as it already exists
Skipping Vaporetto_E_US_PAT23_Rosita_15_Nov_215pmET_Patient as it already exists
Skipping Vaporetto_E_US_PAT24_Rosita_16Nov_2PMEST_Patient as it already exists
Skipping Vaporetto_E_US_PAT29_Rosita_14Nov_5pmET_Nonuser as it already exists
Skipping Vaporetto_E_US_PAT30_Rosita_16_Nov_11amET_Patient as it already exists
Skipping Vaporetto_E_US_PAT31_Rosita_1

In [0]:
%pip list

Package                        Version
------------------------------ -------------
absl-py                        1.0.0
accelerate                     0.25.0
aiohttp                        3.9.1
aiosignal                      1.3.1
anyio                          3.5.0
appdirs                        1.4.4
argon2-cffi                    21.3.0
argon2-cffi-bindings           21.2.0
astor                          0.8.1
asttokens                      2.0.5
astunparse                     1.6.3
async-timeout                  4.0.3
attrs                          22.1.0
audioread                      3.0.1
azure-cognitiveservices-speech 1.33.0
azure-core                     1.29.1
azure-cosmos                   4.3.1
azure-storage-blob             12.19.0
azure-storage-file-datalake    12.14.0
backcall                       0.2.0
bcrypt                         3.2.0
beautifulsoup4                 4.11.1
black                          22.6.0
bleach                         4.1.0
blinker         